In [1]:
import pandas as pd
import json
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

pd.options.display.max_columns = None

In [9]:
## Function to extract artist names
# text: str - string representation of list of dicts of length equal to number of artists
def get_artist(text):
    artists = ast.literal_eval(text)  # list (artists) containing (artists) dictionaries

    return ", ".join(artist['name'] for artist in artists)

# example to test
get_artist("[{'external_urls': {'spotify': 'https://open.spotify.com/artist/1Cs0zKBU1kc0i8ypK3B9ai'}, 'href': 'https://api.spotify.com/v1/artists/1Cs0zKBU1kc0i8ypK3B9ai', 'id': '1Cs0zKBU1kc0i8ypK3B9ai', 'name': 'David Guetta', 'type': 'artist', 'uri': 'spotify:artist:1Cs0zKBU1kc0i8ypK3B9ai'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/64M6ah0SkkRsnPGtGiRAbb'}, 'href': 'https://api.spotify.com/v1/artists/64M6ah0SkkRsnPGtGiRAbb', 'id': '64M6ah0SkkRsnPGtGiRAbb', 'name': 'Bebe Rexha', 'type': 'artist', 'uri': 'spotify:artist:64M6ah0SkkRsnPGtGiRAbb'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/1vyhD5VmyZ7KMfW5gqLgo5'}, 'href': 'https://api.spotify.com/v1/artists/1vyhD5VmyZ7KMfW5gqLgo5', 'id': '1vyhD5VmyZ7KMfW5gqLgo5', 'name': 'J Balvin', 'type': 'artist', 'uri': 'spotify:artist:1vyhD5VmyZ7KMfW5gqLgo5'}]")

'David Guetta, Bebe Rexha, J Balvin'

In [40]:
## Load and clean the data
# index_col=0 : consider first column as index
# sort by position in ascending order and reset index
df = pd.read_csv('played_out.csv', index_col=0).sort_values(by='position', ascending=True).reset_index(drop=True)

# track.type : always same value 'track'
# track.is_local : always same value 'local'
# track.track_number : position on its original album (not relevant for our analysis)
df.drop(columns = ['track.type', 'track.is_local', 'track.track_number'], inplace=True)

# extract artist names from 'track.artists' column
df['artist'] = df['track.artists'].apply(get_artist)

# drop the original 'track.artists' column
df.drop(columns = ['track.artists'], inplace=True)

df.rename(columns={'track.duration_ms':'duration_ms', 'track.explicit':'explicit', 'track.id':'track_id', 'track.name':'track_name', 'track.popularity':'popularity',
                'accousticness':'acousticness'}, inplace=True)

In [14]:
df.to_csv('played_out_cleaned.csv')

print(df.shape)
df.head()


(4572, 23)


,position,played_at,duration_ms,explicit,track_id,track_name,popularity,first_genre,user,gender,age,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,artist
0,0,2025-10-1T01:00:00:000Z,383972,False,7p6oXzBSPAXXz8Xb8gBPki,Subzero - Original Mix,45,3,19befa2d20a544b3819a1236cf7536c4_Harsh1,F,20,0.58800,0.793,0.608,0.8610,6.0,0.0991,-11.638,0.0,0.0673,124.988,0.118,Ben Klock
1,0,2025-10-1T01:00:00:000Z,254200,False,3ENHpbTuY72FukZbwGP6bc,Tear Away,67,4,24bebb12512f476a8e08b14a1aaa08ea_Hannah,F,40,0.00055,0.611,0.648,0.0028,0.0,0.0846,-4.579,1.0,0.0441,132.983,0.457,Drowning Pool
2,0,2025-10-1T01:00:00:000Z,254466,False,6nz35DNIzbtj5ztpDEcW1j,"Kick, Push",64,2,6016a0c171f34d44bc5840e0f521a034_D,F,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lupe Fiasco
3,0,2025-10-1T01:00:00:000Z,383972,False,7p6oXzBSPAXXz8Xb8gBPki,Subzero - Original Mix,45,3,6f0f1e2e101f4f208ede37b241d6fc4c_31owtsxu6xlbj...,M,20,0.58800,0.793,0.608,0.8610,6.0,0.0991,-11.638,0.0,0.0673,124.988,0.118,Ben Klock
4,0,2025-10-1T01:00:00:000Z,223760,False,1lK5iIMKifrxERzS3iimJH,Breathe on Me,56,1,f9a2b3c8d4e1f7a0b5c6d9e2f1a3b8c,M,20,0.00209,0.709,0.565,0.0125,5.0,0.0901,-6.395,0.0,0.0471,112.203,0.571,Britney Spears


In [20]:
# print(df['position'].value_counts().sort_index())
# print(df['position'].value_counts().sort_index().head(10))
print(df['position'].value_counts().sort_index().tail(10))

# positions where it does not have count as 89
for pos in range(df['position'].nunique()):
    if df['position'].value_counts().sort_index().iloc[pos] != 89:
        print("count not as 89 :-",'position :', pos,',','count :', df['position'].value_counts().sort_index().iloc[pos])


position
42    89
43    89
44    89
45    89
46    89
47    89
48    89
49    89
50    71
51    51
Name: count, dtype: int64
count not as 89 :- position : 50 , count : 71
count not as 89 :- position : 51 , count : 51


Song positions 50 and 51 have just 71 and 51 entries.
Which tells the user have recorded more than 50 songs.

In [ ]:
# should we remove these two rows ??
# or remove the first two rows (songs) of the corresponding users

In [30]:
harsh_data_1 = df[df['user'] == '19befa2d20a544b3819a1236cf7536c4_Harsh1'][['position', 'played_at', 'track_id']].sort_values(by='played_at')

In [32]:
harsh_data_1['track_id'].nunique()

52

In [46]:
df[df['user'] == '19befa2d20a544b3819a1236cf7536c4_Harsh1'].sort_values(by='played_at')

df[(df['position'] != 0)]['user'].nunique()

89

In [51]:
grouped = df.groupby('user').agg({'position':'count'}).reset_index()
grouped.groupby('position').agg({'user':'count'}).reset_index()

,position,user
0,50,19
1,51,18
2,52,52


In [ ]:
df.keys()

Index(['position', 'played_at', 'duration_ms', 'explicit', 'track_id',
       'track_name', 'popularity', 'first_genre', 'user', 'gender', 'age',
       'acousticness', 'danceability', 'energy', 'instrumentalness', 'key',
       'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'valence',
       'artist'],
      dtype='object')

In [ ]:
pd.set_option('display.max_rows', None)

df[['user', 'first_genre', 'position', 'track_name']].sort_values(by=['first_genre', 'user', 'position']).reset_index(drop=True)

,user,first_genre,position,track_name
0,23aada40ba9e4409a6609a7efe0343fc_Julie Andrews,1,0,Breathe on Me
1,23aada40ba9e4409a6609a7efe0343fc_Julie Andrews,1,1,As It Was
2,23aada40ba9e4409a6609a7efe0343fc_Julie Andrews,1,2,Watermelon Sugar
3,23aada40ba9e4409a6609a7efe0343fc_Julie Andrews,1,3,Flowers
4,23aada40ba9e4409a6609a7efe0343fc_Julie Andrews,1,4,End of Beginning
...,...,...,...,...
4567,f47677960ad044819291b0a57cae6c84_Ömercan3,5,47,Fur Elise
4568,f47677960ad044819291b0a57cae6c84_Ömercan3,5,48,Days Of Wine And Roses
4569,f47677960ad044819291b0a57cae6c84_Ömercan3,5,49,Dream A Little Dream Of Me
4570,f47677960ad044819291b0a57cae6c84_Ömercan3,5,50,"Tchaikovsky: Swan Lake, Op. 20, Act 2: No. 10,..."
